In [1]:
import nglview
import ipywidgets
import os
import zipfile

## Step 1: Protein and ligand selection

In [2]:
# DOI: 10.2210/pdb3HTB/pdb
# 2-propylphenol in complex with T4 lysozyme
pdbCode = "3HTB"
ligandCode = "JZ4"

mol_charge = 0

proteinFile = pdbCode+'.pdb'
ligandFile = ligandCode+'.pdb'
complexFile = pdbCode+'_'+ligandCode+'.pdb'

In [20]:
gmx_path = '/usr/local/gromacs/bin/gmx' # latest local gromacs installation

## Step 2: Downloading Protein

From: RCSB PDB database

In [3]:
pdb_output = pdbCode+'.orig.pdb'

In [5]:
from biobb_io.api.pdb import pdb

prop = {
    'pdb_code': pdbCode,
    'filter': False
}

# Downloading 3HTB from the Protein Data Bank
pdb(output_pdb_path=pdb_output, properties=prop)

2021-11-02 21:36:14,766 [MainThread  ] [INFO ]  Downloading: 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent


/home/dhnchandan/anaconda3/envs/nb_md_analysis/lib/python3.7/site-packages/urllib3/connectionpool.py:1020: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2021-11-02 21:36:17,580 [MainThread  ] [INFO ]  Writting pdb to: 3HTB.orig.pdb


0

## Step 3: Extracting Protein, Ligand and Protein-Ligand Complex

In [4]:
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb

prop = {
    'heteroatoms' : [{"name": ligandCode}]
}

extract_heteroatoms(
    input_structure_path=pdb_output,
    output_heteroatom_path=ligandFile,
    properties=prop
)

extract_molecule(
    input_structure_path=pdb_output,
    output_molecule_path=proteinFile
)

# Concat protein and igand into one file
cat_pdb(
    input_structure1=proteinFile,
    input_structure2=ligandFile,
    output_structure_path=complexFile
)

print(proteinFile, ligandFile, complexFile)

## Step 4: Visualizing 3D structures

In [7]:
view_protein = nglview.show_structure_file(proteinFile)
view_protein._remote_call('setSize', target='Widget', args=['350px','400px'])
view_protein.camera='orthographic'
view_protein

view_ligand = nglview.show_structure_file(ligandFile)
view_ligand.add_representation(repr_type='ball+stick')
view_ligand._remote_call('setSize', target='Widget', args=['350px','400px'])
view_ligand.camera='orthographic'
view_ligand

view_complex = nglview.show_structure_file(complexFile)
view_complex.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view_complex._remote_call('setSize', target='Widget', args=['350px','400px'])
view_complex.camera='orthographic'
view_complex

ipywidgets.HBox([view_protein, view_ligand, view_complex])

## Step 5: Fixing protein structure

- missing side-chain atoms
- missing backbone atoms, heteroatoms and modified residues

In [4]:
fixed_pdb_out = pdbCode + '_fixed.pdb'

In [8]:
from biobb_model.model.fix_side_chain import fix_side_chain

fix_side_chain(input_pdb_path=proteinFile, output_pdb_path=fixed_pdb_out)

2021-11-02 22:06:01,503 [MainThread  ] [INFO ]  Not using any container
2021-11-02 22:06:01,503 [MainThread  ] [INFO ]  check_structure -i 3HTB.pdb -o 3HTB_fixed.pdb --force_save fixside --fix ALL

2021-11-02 22:06:01,900 [MainThread  ] [INFO ]  Exit code 0

2021-11-02 22:06:01,901 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure 3HTB.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  163
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1300

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Fina

0

## Step 6: Creating protein topology

- Force field: amber99sb-ildn
- Water molecule type: spc/e

In [5]:
output_pdb2gmx_gro = pdbCode+'_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode+'_pdb2gmx_top.zip'

In [10]:
from biobb_md.gromacs.pdb2gmx import pdb2gmx

prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce',
#     'gmx_path': gmx_path
}

pdb2gmx(
    input_pdb_path=fixed_pdb_out,
    output_gro_path=output_pdb2gmx_gro,
    output_top_zip_path=output_pdb2gmx_top_zip,
    properties=prop
)

2021-11-02 22:13:29,172 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20212 version detected
2021-11-02 22:13:29,173 [MainThread  ] [INFO ]  Not using any container
2021-11-02 22:13:29,174 [MainThread  ] [INFO ]  /usr/local/gromacs/bin/gmx -nobackup -nocopyright pdb2gmx -f 3HTB_fixed.pdb -o 3HTB_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2021-11-02 22:13:30,780 [MainThread  ] [INFO ]  Exit code 0

2021-11-02 22:13:30,781 [MainThread  ] [INFO ]  Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b

going to rename amber99sb-ildn.ff/dna.r2b

going to rename amber99sb-ildn.ff/rna.r2b
Reading 3HTB_fixed.pdb...
Read '', 1364 atoms

Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.

There are 1 chains and 0 blocks of water and 163 residues with 1364 atoms

  chain  #res #atoms

  1 'A'   163   1364  

there were 0 atoms with zero occupancy and 178 atoms with          occu

0

## Step 7: Preparing ligand

- Force field: amberGAFF


### Step 7.a: Add hydrogen atom

In [6]:
output_reduce_h = ligandCode+'.reduce.H.pdb'

In [6]:
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

prop = {
    'nuclear' : 'true'
}

reduce_add_hydrogens(
    input_path=ligandFile,
    output_path=output_reduce_h,
    properties=prop
)

2021-11-03 06:04:29,096 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:04:29,097 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.pdb > JZ4.reduce.H.pdb

2021-11-03 06:04:29,236 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:04:29,237 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "/home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.pdb"
Database of HETATM connections: "/home/dhnchandan/anaconda3/envs/nb_md_analysis//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Si

0

### Step 7.b: Energetically minimize the system

In [7]:
output_babel_min = ligandCode+'.H.min.mol2'

In [7]:
from biobb_chemistry.babelm.babel_minimize import babel_minimize

prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}

babel_minimize(
    input_path=output_reduce_h,
    output_path=output_babel_min,
    properties=prop
)

2021-11-03 06:06:19,699 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2021-11-03 06:06:19,700 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2021-11-03 06:06:19,701 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2021-11-03 06:06:19,701 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2021-11-03 06:06:19,702 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2021-11-03 06:06:19,703 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2021-11-03 06:06:19,703 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:06:19,704 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.reduce.H.pdb -omol2 > JZ4.H.min.mol2

2021-11-03 06:06:20,287 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:06:20,288 [MainThread  ] [INFO ]  
A T O M   T

0

### Step 7.c: Visualizing 3D structures

In [8]:
# Original structure view
view_original = nglview.show_structure_file(ligandFile)
view_original.add_representation(repr_type='ball+stick')
view_original._remote_call('setSize', target='Widget', args=['350px','400px'])
view_original.camera='orthographic'
view_original

# After adding hydrogen view
view_h_added = nglview.show_structure_file(output_reduce_h)
view_h_added.add_representation(repr_type='ball+stick')
view_h_added._remote_call('setSize', target='Widget', args=['350px','400px'])
view_h_added.camera='orthographic'
view_h_added

# After energy minimization view
view_e_minimized = nglview.show_structure_file(output_babel_min)
view_e_minimized.add_representation(repr_type='ball+stick')
view_e_minimized._remote_call('setSize', target='Widget', args=['350px','400px'])
view_e_minimized.camera='orthographic'
view_e_minimized

ipywidgets.HBox([view_original, view_h_added, view_e_minimized])

### Step 7.d: Generating ligand topology

In [9]:
output_acpype_gro = ligandCode+'params.gro'
output_acpype_itp = ligandCode+'params.itp'
output_acpype_top = ligandCode+'params.top'
output_acpype = ligandCode+'params'

In [9]:
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

acpype_params_gmx(
    input_path=output_babel_min, 
    output_path_gro=output_acpype_gro,
    output_path_itp=output_acpype_itp,
    output_path_top=output_acpype_top,
    properties=prop
)

2021-11-03 06:12:26,193 [MainThread  ] [INFO ]  Not using any container
2021-11-03 06:12:26,194 [MainThread  ] [INFO ]  acpype -i /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/JZ4.H.min.mol2 -b JZ4params.U15Loq -n 0

2021-11-03 06:12:27,738 [MainThread  ] [INFO ]  Exit code 0

2021-11-03 06:12:27,739 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2021 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'JZ4'....
Checking parameters for unit 'JZ4'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
==> * Tleap OK *
==> Removing temporary files...
==> Writing NE

0

## Step 8: Preparing Ligand Restraints

In subsequent steps of the pipeline, such as the equilibration stages of the protein-ligand complex system, it is recommended to apply some restraints to the small molecule, to avoid a possible change in position due to protein repulsion. Position restraints will be applied to the ligand, using a force constant of 1000 KJ/mol*nm^2 on the three coordinates: x, y and z. In this steps the restriction files will be created and integrated in the ligand topology.


### Step 8.a: Creating an index file for the small molecule heavy atoms

In [10]:
output_ligand_ndx = ligandCode+'_index.ndx'

In [38]:
from biobb_md.gromacs.make_ndx import make_ndx

prop = {
    'selection': "0 & ! a H*",
#     'gmx_path': gmx_path
}

make_ndx(
    input_structure_path=output_acpype_gro,
    output_ndx_path=output_ligand_ndx,
    properties=prop
)

2021-11-04 06:37:35,956 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2021-11-04 06:37:35,957 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:37:35,958 [MainThread  ] [INFO ]  echo -e '0 & ! a H*\nq' | gmx -nobackup -nocopyright make_ndx -f JZ4params.gro -o JZ4_index.ndx

2021-11-04 06:37:35,990 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:37:35,991 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :    22 atoms
  1 Other               :    22 atoms
  2 JZ4                 :    22 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 "

0

### Step 8.b: Generating the position restraints file

In [12]:
output_restraints_top = ligandCode+'_posres.itp'

In [39]:
from biobb_md.gromacs.genrestr import genrestr

prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "System",
#     'gmx_path': gmx_path
}

genrestr(
    input_structure_path=output_acpype_gro,
    input_ndx_path=output_ligand_ndx,
    output_itp_path=output_restraints_top,
    properties=prop
)

2021-11-04 06:37:44,477 [MainThread  ] [INFO ]  GROMACS Genrestr 20191 version detected
2021-11-04 06:37:44,478 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:37:44,480 [MainThread  ] [INFO ]  echo "System" | gmx -nobackup -nocopyright genrestr -f JZ4params.gro -o JZ4_posres.itp -fc 1000 1000 1000 -n JZ4_index.ndx

2021-11-04 06:37:44,495 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:37:44,496 [MainThread  ] [INFO ]  Select group to position restrain
Selected 0: 'System'

2021-11-04 06:37:44,497 [MainThread  ] [INFO ]                       :-) GROMACS - gmx genrestr, 2019.1 (-:

Executable:   /home/dhnchandan/anaconda3/envs/nb_md_analysis/bin/gmx
Data prefix:  /home/dhnchandan/anaconda3/envs/nb_md_analysis
Working dir:  /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks
Command line:
  gmx -nobackup -nocopyright genrestr -f JZ4params.gro -o JZ4_posres.itp -fc 1000 1000 1000 -n JZ4_index.ndx


Reading structure file
Group     0 (         System) h

0

## Step 9: Creating protein-ligand complex structure file


Building new protein-ligand complex PDB file with:
- The new protein system with fixed problems from Step 5 (Fixing protein structure step) and hydrogens atoms added from Step 6 (Creating protein topology step).
- The new ligand system with hydrogens atoms added from Step 7 (Preparing ligand step).


In [14]:
proteinFile_H = pdbCode+'_'+ligandCode+'_complex_H.pdb'
ligandFile_H = ligandCode+'_complex_H.pdb'
complexFile_H = pdbCode+'_'+ligandCode+'_H.pdb'

In [40]:
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
prop = {
    'selection' : 'System',
#     'gmx_path': gmx_path
}

gmx_trjconv_str(
    input_structure_path=output_pdb2gmx_gro,
    input_top_path=output_pdb2gmx_gro,
    output_str_path=proteinFile_H,
    properties=prop,
)

# Convert gro (with hydrogens) to pdb (LIGAND)
prop = {
    'selection' : 'System',
#     'gmx_path': gmx_path
}

gmx_trjconv_str(
    input_structure_path=output_acpype_gro,
    input_top_path=output_acpype_gro,
    output_str_path=ligandFile_H, 
    properties=prop
)


# Concatenating both PDB files: Protein + Ligand
cat_pdb(
    input_structure1=proteinFile_H,
    input_structure2=ligandFile_H,
    output_structure_path=complexFile_H
)

2021-11-04 06:37:52,956 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:37:52,957 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_pdb2gmx.gro -s /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_pdb2gmx.gro -o 3HTB_JZ4_complex_H.pdb

2021-11-04 06:37:52,998 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:37:53,000 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2021-11-04 06:37:53,000 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  G

0

## Step 10: Creating topology file for new protein-ligand complex

In [16]:
output_complex_top = pdbCode+'_'+ligandCode+'_complex.top.zip'
posresifdef = "POSRES_"+ligandCode.upper()

In [17]:
from biobb_md.gromacs_extra.append_ligand import append_ligand

prop = {
    'posres_name': posresifdef
}

append_ligand(
    input_top_zip_path=output_pdb2gmx_top_zip,
    input_posres_itp_path=output_restraints_top,
    input_itp_path=output_acpype_itp, 
    output_top_zip_path=output_complex_top,
    properties=prop
)

2021-11-04 05:55:23,008 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_pdb2gmx_top.zip
2021-11-04 05:55:23,009 [MainThread  ] [INFO ]  to:
2021-11-04 05:55:23,010 [MainThread  ] [INFO ]  ['e0d55218-3a3c-4309-8a5b-3882b91a5430/p2g.top', 'e0d55218-3a3c-4309-8a5b-3882b91a5430/posre.itp']
2021-11-04 05:55:23,010 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 05:55:23,011 [MainThread  ] [INFO ]  3HTB_pdb2gmx_top.zip
2021-11-04 05:55:23,012 [MainThread  ] [INFO ]  To: 
2021-11-04 05:55:23,012 [MainThread  ] [INFO ]  e0d55218-3a3c-4309-8a5b-3882b91a5430/p2g.top
2021-11-04 05:55:23,013 [MainThread  ] [INFO ]  e0d55218-3a3c-4309-8a5b-3882b91a5430/posre.itp
2021-11-04 05:55:23,041 [MainThread  ] [INFO ]  Compressing topology to: 3HTB_JZ4_complex.top.zip
2021-11-04 05:55:23,043 [MainThread  ] [INFO ]  Ignored file e0d55218-3a3c-4309-8a5b-3882b91a5430/amber99sb-ildn.ff/forcefield.itp
2021-11-04 05:55:23,058 [MainThread  ] [INFO ]  Ig

0

## Step 11: Creating solvent box


Define the unit cell for the protein-ligand complex to fill it with water molecules.

- Box type: Truncated octahedron (best for spheric globular protein)
- Distance: 0.8 nm (protein to box)
- Protein postion: centered in the box

In [18]:
output_editconf_gro = pdbCode+'_'+ligandCode+'_complex_editconf.gro'

In [41]:
from biobb_md.gromacs.editconf import editconf

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 0.8,
#     'gmx_path': gmx_path
}

editconf(
    input_gro_path=complexFile_H, 
    output_gro_path=output_editconf_gro,
    properties=prop
)

2021-11-04 06:38:02,752 [MainThread  ] [INFO ]  Centering molecule in the box.
2021-11-04 06:38:02,753 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.80
2021-11-04 06:38:02,754 [MainThread  ] [INFO ]  Box type: octahedron
2021-11-04 06:38:02,754 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2021-11-04 06:38:02,755 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:38:02,756 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f 3HTB_JZ4_H.pdb -o 3HTB_JZ4_complex_editconf.gro -d 0.8 -bt octahedron -c

2021-11-04 06:38:02,808 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:38:02,809 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 2636 atoms
Volume: 1322.64 nm^3, corresponds to roughly 595100 electrons
No velocities found
    system size :  4.069  4.192  5.073 (nm)
    diameter    :  5.849               (nm)
    center      :  2.241 -1.669 -0.935 (nm)
    box vec

0

## Step 12: Filling solvent box with water molecules

- Water model: SPC (equilibrated 3-point solvent model)

In [27]:
output_solvate_gro = pdbCode+'_'+ligandCode+'_solvate.gro'
output_solvate_top_zip = pdbCode+'_'+ligandCode+'_solvate_top.zip'

In [42]:
from biobb_md.gromacs.solvate import solvate

prop = {
#     'gmx_path': gmx_path
}

solvate(
    input_solute_gro_path=output_editconf_gro,
    output_gro_path=output_solvate_gro,
    input_top_zip_path=output_complex_top,
    output_top_zip_path=output_solvate_top_zip,
    properties=prop
)

2021-11-04 06:38:16,560 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_complex.top.zip
2021-11-04 06:38:16,562 [MainThread  ] [INFO ]  to:
2021-11-04 06:38:16,563 [MainThread  ] [INFO ]  ['f4f57e82-d94a-48ed-a37b-bb40f75acb24/JZ4_posres.itp', 'f4f57e82-d94a-48ed-a37b-bb40f75acb24/JZ4params.itp', 'f4f57e82-d94a-48ed-a37b-bb40f75acb24/ligand.top', 'f4f57e82-d94a-48ed-a37b-bb40f75acb24/posre.itp']
2021-11-04 06:38:16,564 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 06:38:16,566 [MainThread  ] [INFO ]  3HTB_JZ4_complex.top.zip
2021-11-04 06:38:16,567 [MainThread  ] [INFO ]  To: 
2021-11-04 06:38:16,567 [MainThread  ] [INFO ]  f4f57e82-d94a-48ed-a37b-bb40f75acb24/JZ4_posres.itp
2021-11-04 06:38:16,568 [MainThread  ] [INFO ]  f4f57e82-d94a-48ed-a37b-bb40f75acb24/JZ4params.itp
2021-11-04 06:38:16,569 [MainThread  ] [INFO ]  f4f57e82-d94a-48ed-a37b-bb40f75acb24/ligand.top
2021-11-04 06:38:16,569 [MainThread  ] [INFO ]  f4f5

0

## Step 13: Visualizing 3D structure

In [30]:
#Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'

view.render_image()
view.download_image(filename='solvent_box.png')

view

NGLWidget()

## Step 14: Adding ions


Add ions to neutralize the protein-ligand complex and reach a desired ionic concentration.

- Creating portable binary run file for ion generation
- Adding ions to neutralize the system and reach a 0.05 molar ionic concentration

### Step 14.a: Creating binary run file

In [35]:
output_gppion_tpr = pdbCode+'_'+ligandCode+'_complex_gppion.tpr'

In [43]:
from biobb_md.gromacs.grompp import grompp

prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'minimization',
    'maxwarn': 1,
#     'gmx_path': gmx_path
}

grompp(
    input_gro_path=output_solvate_gro,
    input_top_zip_path=output_solvate_top_zip, 
    output_tpr_path=output_gppion_tpr,
    properties=prop
)

2021-11-04 06:38:23,768 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_solvate_top.zip
2021-11-04 06:38:23,770 [MainThread  ] [INFO ]  to:
2021-11-04 06:38:23,770 [MainThread  ] [INFO ]  ['fe7be492-16a0-45a1-bd35-6ca44b2b708d/JZ4_posres.itp', 'fe7be492-16a0-45a1-bd35-6ca44b2b708d/JZ4params.itp', 'fe7be492-16a0-45a1-bd35-6ca44b2b708d/ligand.top', 'fe7be492-16a0-45a1-bd35-6ca44b2b708d/posre.itp']
2021-11-04 06:38:23,771 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 06:38:23,772 [MainThread  ] [INFO ]  3HTB_JZ4_solvate_top.zip
2021-11-04 06:38:23,772 [MainThread  ] [INFO ]  To: 
2021-11-04 06:38:23,773 [MainThread  ] [INFO ]  fe7be492-16a0-45a1-bd35-6ca44b2b708d/JZ4_posres.itp
2021-11-04 06:38:23,774 [MainThread  ] [INFO ]  fe7be492-16a0-45a1-bd35-6ca44b2b708d/JZ4params.itp
2021-11-04 06:38:23,774 [MainThread  ] [INFO ]  fe7be492-16a0-45a1-bd35-6ca44b2b708d/ligand.top
2021-11-04 06:38:23,775 [MainThread  ] [INFO ]  fe7b

0

### Step 14.b: Neutralizing the system

In [33]:
output_genion_gro = pdbCode+'_'+ligandCode+'_genion.gro'
output_genion_top_zip = pdbCode+'_'+ligandCode+'_genion_top.zip'

In [44]:
from biobb_md.gromacs.genion import genion

prop={
    'neutral':True,
    'concentration':0.05,
#     'gmx_path': gmx_path
}

genion(
    input_tpr_path=output_gppion_tpr,
    output_gro_path=output_genion_gro, 
    input_top_zip_path=output_solvate_top_zip,
    output_top_zip_path=output_genion_top_zip, 
    properties=prop
)

2021-11-04 06:41:01,580 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_solvate_top.zip
2021-11-04 06:41:01,581 [MainThread  ] [INFO ]  to:
2021-11-04 06:41:01,582 [MainThread  ] [INFO ]  ['f1dbce03-5b02-4273-90f2-e238f01af1da/JZ4_posres.itp', 'f1dbce03-5b02-4273-90f2-e238f01af1da/JZ4params.itp', 'f1dbce03-5b02-4273-90f2-e238f01af1da/ligand.top', 'f1dbce03-5b02-4273-90f2-e238f01af1da/posre.itp']
2021-11-04 06:41:01,583 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 06:41:01,584 [MainThread  ] [INFO ]  3HTB_JZ4_solvate_top.zip
2021-11-04 06:41:01,584 [MainThread  ] [INFO ]  To: 
2021-11-04 06:41:01,585 [MainThread  ] [INFO ]  f1dbce03-5b02-4273-90f2-e238f01af1da/JZ4_posres.itp
2021-11-04 06:41:01,585 [MainThread  ] [INFO ]  f1dbce03-5b02-4273-90f2-e238f01af1da/JZ4params.itp
2021-11-04 06:41:01,586 [MainThread  ] [INFO ]  f1dbce03-5b02-4273-90f2-e238f01af1da/ligand.top
2021-11-04 06:41:01,587 [MainThread  ] [INFO ]  f1db

0

## Step 15: Visualizing 3D structure

In [45]:
#Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'

view.render_image()
view.download_image(filename='step15_protein_ligand_complex.png')

view

NGLWidget()

## Step 16: Energetically minimize the system

Energetically minimize the protein-ligand complex till reaching a desired potential energy.

- Creating portable binary run file for energy minimization
- Energetically minimize the protein-ligand complex till reaching a force of 500 kJ/mol/nm.
- Checking energy minimization results. Plotting energy by time during the minimization process.

### Step 16.a: Creating portable binary run file

In [46]:
output_gppmin_tpr = pdbCode+'_'+ligandCode+'_gppmin.tpr'

In [47]:
from biobb_md.gromacs.grompp import grompp

prop = {
    'mdp':{
        'nsteps':'5000',
        'emstep': 0.01,
        'emtol':'500'
    },
    'simulation_type':'minimization',
#     'gmx_path': gmx_path
}

grompp(
    input_gro_path=output_genion_gro,
    input_top_zip_path=output_genion_top_zip,
    output_tpr_path=output_gppmin_tpr,
    properties=prop
)

2021-11-04 06:53:05,437 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_genion_top.zip
2021-11-04 06:53:05,440 [MainThread  ] [INFO ]  to:
2021-11-04 06:53:05,441 [MainThread  ] [INFO ]  ['9527cae1-facd-4e38-96ff-758aabc7ceaf/JZ4_posres.itp', '9527cae1-facd-4e38-96ff-758aabc7ceaf/JZ4params.itp', '9527cae1-facd-4e38-96ff-758aabc7ceaf/ligand.top', '9527cae1-facd-4e38-96ff-758aabc7ceaf/posre.itp']
2021-11-04 06:53:05,442 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 06:53:05,442 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2021-11-04 06:53:05,443 [MainThread  ] [INFO ]  To: 
2021-11-04 06:53:05,443 [MainThread  ] [INFO ]  9527cae1-facd-4e38-96ff-758aabc7ceaf/JZ4_posres.itp
2021-11-04 06:53:05,445 [MainThread  ] [INFO ]  9527cae1-facd-4e38-96ff-758aabc7ceaf/JZ4params.itp
2021-11-04 06:53:05,446 [MainThread  ] [INFO ]  9527cae1-facd-4e38-96ff-758aabc7ceaf/ligand.top
2021-11-04 06:53:05,446 [MainThread  ] [INFO ]  9527ca

0

### Step 16.b: Energetically minimize the protein-ligand complex

In [48]:
output_min_trr = pdbCode+'_'+ligandCode+'_min.trr'
output_min_gro = pdbCode+'_'+ligandCode+'_min.gro'
output_min_edr = pdbCode+'_'+ligandCode+'_min.edr'
output_min_log = pdbCode+'_'+ligandCode+'_min.log'

In [49]:
from biobb_md.gromacs.mdrun import mdrun

prop = {
#     'gmx_path': gmx_path
}

mdrun(
    input_tpr_path=output_gppmin_tpr,
    output_trr_path=output_min_trr, 
    output_gro_path=output_min_gro,
    output_edr_path=output_min_edr, 
    output_log_path=output_min_log,
    properties=prop
)

2021-11-04 06:53:17,548 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2021-11-04 06:53:17,550 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:53:17,551 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmin.tpr -o 3HTB_JZ4_min.trr -c 3HTB_JZ4_min.gro -e 3HTB_JZ4_min.edr -g 3HTB_JZ4_min.log

2021-11-04 06:54:12,390 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:54:12,391 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/dhnchandan/anaconda3/envs/nb_md_analysis/bin/gmx
Data prefix:  /home/dhnchandan/anaconda3/envs/nb_md_analysis
Working dir:  /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmin.tpr -o 3HTB_JZ4_min.trr -c 3HTB_JZ4_min.gro -e 3HTB_JZ4_min.edr -g 3HTB_JZ4_min.log

Compiled SIMD: SSE2, but for this host/run AVX2_128 might be better (see log).
The current CPU can measure timings more ac

0

### Step 16.c: Plotting em result

In [50]:
output_min_ene_xvg = pdbCode+'_'+ligandCode+'_min_ene.xvg'

In [51]:
from biobb_analysis.gromacs.gmx_energy import gmx_energy

prop = {
    'terms':  ["Potential"],
#     'gmx_path': gmx_path
}

gmx_energy(
    input_energy_path=output_min_edr, 
    output_xvg_path=output_min_ene_xvg, 
    properties=prop
)

2021-11-04 06:54:30,654 [MainThread  ] [INFO ]  Not using any container
2021-11-04 06:54:30,655 [MainThread  ] [INFO ]  gmx energy -f /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_min.edr -o 3HTB_JZ4_min_ene.xvg -xvg none < d4615c9b-d540-4fea-939e-3b15250a993f/instructions.in

2021-11-04 06:54:30,673 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 06:54:30,675 [MainThread  ] [INFO ]  
Statistics over 1749 steps [ 0.0000 through 1748.0000 ps ], 1 data sets
All statistics are over 1384 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -529969       8700    22444.2   -56940.6  (kJ/mol)

2021-11-04 06:54:30,675 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Be

0

In [52]:
import plotly
import plotly.graph_objs as go

# Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg, 'r') as energy_file:
    x, y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
})

plotly.offline.iplot(fig)

## Step 17: Equilibrating the system (NVT)

Equilibrate the protein-ligand complex system in NVT ensemble (constant Number of particles, Volume and Temperature).

- Creating an index file with a new group including the protein-ligand complex.
- Creating portable binary run file for system equilibration
- Equilibrate the protein-ligand complex with NVT ensemble.
- Checking NVT Equilibration results. Plotting system temperature by time during the NVT equilibration process.

### Step 17.a: Creating an index file

In [53]:
output_complex_ndx = pdbCode+'_'+ligandCode+'_index.ndx'

In [54]:
from biobb_md.gromacs.make_ndx import make_ndx

prop = {
    'selection': "\"Protein\"|\"Other\"",
#     'gmx_path': gmx_path
}

make_ndx(
    input_structure_path=output_min_gro,
    output_ndx_path=output_complex_ndx,
    properties=prop
)

2021-11-04 07:05:11,396 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2021-11-04 07:05:11,396 [MainThread  ] [INFO ]  Not using any container
2021-11-04 07:05:11,397 [MainThread  ] [INFO ]  echo -e '"Protein"|"Other"\nq' | gmx -nobackup -nocopyright make_ndx -f 3HTB_JZ4_min.gro -o 3HTB_JZ4_index.ndx

2021-11-04 07:05:11,512 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 07:05:11,512 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:   163    Protein residues
There are:     1      Other residues
There are:  9505      Water residues
There are:    26        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 31177 atoms
  1 Protein             :  2614 atoms
  2 Protein-H           :  1301 atoms
  3 C-alpha             :   163 atoms
  4 Backbone   

0

### Step 17.b: Creating portable binary run file

In [55]:
output_gppnvt_tpr = pdbCode+'_'+ligandCode+'gppnvt.tpr'

In [56]:
from biobb_md.gromacs.grompp import grompp

prop = {
    'mdp':{
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'nvt',
#     'gmx_path': gmx_path
}

grompp(
    input_gro_path=output_min_gro,
    input_top_zip_path=output_genion_top_zip,
    input_ndx_path=output_complex_ndx,
    output_tpr_path=output_gppnvt_tpr,
    properties=prop
)

2021-11-04 07:05:26,245 [MainThread  ] [INFO ]  Extracting: /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_genion_top.zip
2021-11-04 07:05:26,247 [MainThread  ] [INFO ]  to:
2021-11-04 07:05:26,248 [MainThread  ] [INFO ]  ['8f818254-e317-43c0-a0f4-67aae645be14/JZ4_posres.itp', '8f818254-e317-43c0-a0f4-67aae645be14/JZ4params.itp', '8f818254-e317-43c0-a0f4-67aae645be14/ligand.top', '8f818254-e317-43c0-a0f4-67aae645be14/posre.itp']
2021-11-04 07:05:26,248 [MainThread  ] [INFO ]  Unzipping: 
2021-11-04 07:05:26,249 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2021-11-04 07:05:26,250 [MainThread  ] [INFO ]  To: 
2021-11-04 07:05:26,250 [MainThread  ] [INFO ]  8f818254-e317-43c0-a0f4-67aae645be14/JZ4_posres.itp
2021-11-04 07:05:26,251 [MainThread  ] [INFO ]  8f818254-e317-43c0-a0f4-67aae645be14/JZ4params.itp
2021-11-04 07:05:26,251 [MainThread  ] [INFO ]  8f818254-e317-43c0-a0f4-67aae645be14/ligand.top
2021-11-04 07:05:26,252 [MainThread  ] [INFO ]  8f8182

0

### Step 17.c: Running NVT equilibration

In [57]:
output_nvt_trr = pdbCode+'_'+ligandCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_'+ligandCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_'+ligandCode+'_nvt.edr'
output_nvt_log = pdbCode+'_'+ligandCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_'+ligandCode+'_nvt.cpt'

In [58]:
from biobb_md.gromacs.mdrun import mdrun

prop = {
#     'gmx_path': gmx_path
}

mdrun(
    input_tpr_path=output_gppnvt_tpr,
    output_trr_path=output_nvt_trr,
    output_gro_path=output_nvt_gro,
    output_edr_path=output_nvt_edr,
    output_log_path=output_nvt_log,
    output_cpt_path=output_nvt_cpt,
    properties=prop
)

2021-11-04 07:05:45,176 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2021-11-04 07:05:45,177 [MainThread  ] [INFO ]  Not using any container
2021-11-04 07:05:45,177 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4gppnvt.tpr -o 3HTB_JZ4_nvt.trr -c 3HTB_JZ4_nvt.gro -e 3HTB_JZ4_nvt.edr -g 3HTB_JZ4_nvt.log -cpo 3HTB_JZ4_nvt.cpt

2021-11-04 07:06:42,918 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 07:06:42,919 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /home/dhnchandan/anaconda3/envs/nb_md_analysis/bin/gmx
Data prefix:  /home/dhnchandan/anaconda3/envs/nb_md_analysis
Working dir:  /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4gppnvt.tpr -o 3HTB_JZ4_nvt.trr -c 3HTB_JZ4_nvt.gro -e 3HTB_JZ4_nvt.edr -g 3HTB_JZ4_nvt.log -cpo 3HTB_JZ4_nvt.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_128 might be better (see log).
T

0

### Step 17.d: Checking NVT Equilibration results

In [59]:
output_nvt_temp_xvg = pdbCode+'_'+ligandCode+'_nvt_temp.xvg'

In [60]:
from biobb_analysis.gromacs.gmx_energy import gmx_energy

prop = {
    'terms':  ["Temperature"],
#     'gmx_path': gmx_path
}

gmx_energy(
    input_energy_path=output_nvt_edr, 
    output_xvg_path=output_nvt_temp_xvg, 
    properties=prop
)

2021-11-04 07:10:34,326 [MainThread  ] [INFO ]  Not using any container
2021-11-04 07:10:34,327 [MainThread  ] [INFO ]  gmx energy -f /home/dhnchandan/Documents/cc/gromacs/nb_md_analysis/project/notebooks/3HTB_JZ4_nvt.edr -o 3HTB_JZ4_nvt_temp.xvg -xvg none < 002b83f0-cbfa-4ddb-99ea-4faaf8c9e6c4/instructions.in

2021-11-04 07:10:34,346 [MainThread  ] [INFO ]  Exit code 0

2021-11-04 07:10:34,347 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 1 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                  298.11        1.6    8.59273    8.44815  (K)

2021-11-04 07:10:34,348 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par B

0

In [61]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg, 'r') as temperature_file:
    x, y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
})

plotly.offline.iplot(fig)